In [1]:
import pandas as pd
import os

data_dir = './data/'
file = 'userid-timestamp-artid-artname-traid-traname.tsv'

In [2]:
df = pd.read_csv(data_dir + file, delimiter="\t", header=None, error_bad_lines=False)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [10]:
df.describe()

,0,1,2,3,4,5
count,19098853,19098853,18498005,19098853,16936134,19098841
unique,992,17454730,107295,173921,960402,1083471
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,183103,248,115099,115099,3991,17561


In [14]:
drop_na_df = df.dropna(axis=0, how='any')

In [18]:
drop_na_df.describe()

,0,1,2,3,4,5
count,16936134,16936134,16936134,16936134,16936134,16936134
unique,992,15631466,83905,81751,960402,693231
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,172042,193,111488,111488,3991,14908
